In [1]:
%%writefile assignment41.cu
// Записываем код  в файл для компиляции CUDA
// Assignment 4 Task 1
// Вычислить суммы элементов массива с  использованием глобальной памяти
// Сравнить результат и время выполнения с  последовательной реализацией на CPU для массива размером 100 000 элементов.

#include <iostream>               // Для стандартный ввод-вывод
#include <cuda_runtime.h>         // Для CUDA Runtime API
#include <chrono>                 // Для измерения времени

using namespace std;              // Чтобы не писать std::

// CUDA KERNEL
__global__ void sumKernel(int *d_array, int *d_result, int n) {                 // Ядро CUDA — выполняется на GPU
    int idx = threadIdx.x + blockIdx.x * blockDim.x;                            // Глобальный индекс потока
    if (idx < n) {                                                              // Каждый поток добавляет один элемент массива в глобальную переменную результата
        atomicAdd(d_result, d_array[idx]);                                      // Атомарное сложение в глобальной памяти
    }
}

// Основная функция
int main() {
    const int N = 100000;               // Размер массива
    const int SIZE = N * sizeof(int);   // Размер массива в байтах

    int *h_array;                       // Указатель на массив в оперативной памяти (host)
    int h_result_cpu = 0;               // Результат суммирования на CPU
    int h_result_gpu = 0;               // Результат суммирования на GPU

    // Выделяем память на CPU
    h_array = new int[N];               // Создаём массив в памяти CPU

    // Инициализация массива
    for (int i = 0; i < N; i++) {
        h_array[i] = 1;                 // Заполняем единицами (сумма должна быть N)
    }                                   // Конец цикла


    auto start_cpu = chrono::high_resolution_clock::now(); // Начало замера CPU

    for (int i = 0; i < N; i++) {      // Последовательный цикл суммирования
        h_result_cpu += h_array[i];    // Прибавляем элемент массива к сумме CPU
    }

    auto end_cpu = chrono::high_resolution_clock::now();     // Конец замера CPU
    chrono::duration<double> cpu_time = end_cpu - start_cpu; // Вычисляем продолжительность времени

    // Выделение памяти на GPU
    int *d_array;                              // Массив на GPU
    int *d_result;                             // Переменная результата на GPU

    cudaMalloc((void**)&d_array, SIZE);        // Выделяем память под массив на GPU
    cudaMalloc((void**)&d_result, sizeof(int));// Выделяем память под результат на GPU

    cudaMemcpy(d_array, h_array, SIZE, cudaMemcpyHostToDevice); // Копируем массив CPU → GPU
    cudaMemset(d_result, 0, sizeof(int));                       // Обнуляем результат на GPU

    // Настройка конфигурации запуска
    int threadsPerBlock = 256;                                        // Количество потоков в блоке
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;  // Количество блоков, чтобы покрыть весь массив

    // Запуск таймера GPU
    cudaEvent_t start_gpu, stop_gpu;        // CUDA-события для измерения времени
    cudaEventCreate(&start_gpu);            // Инициализация событие старта
    cudaEventCreate(&stop_gpu);             // Инициализируем событие конец
    cudaEventRecord(start_gpu);             // Старт GPU таймера

    //  Запуск CUDA-ядра
    sumKernel<<<blocksPerGrid, threadsPerBlock>>>(d_array, d_result, N);


    cudaEventRecord(stop_gpu);              // Остановка GPU таймера
    cudaEventSynchronize(stop_gpu);         // Ждём завершения

    float gpu_time = 0;                     // Переменная для хранения времени GPU
    cudaEventElapsedTime(&gpu_time, start_gpu, stop_gpu); // Время в миллисекундах

    // Копируем результат GPU → CPU
    cudaMemcpy(&h_result_gpu, d_result, sizeof(int), cudaMemcpyDeviceToHost);

    // Вывод результатов
    cout << "CPU результат: " << h_result_cpu << endl;
    cout << "GPU результат: " << h_result_gpu << endl;
    cout << "CPU время: " << cpu_time.count() * 1000 << " мс" << endl;
    cout << "GPU время: " << gpu_time << " мс" << endl;

    // Вычисляем ускорение
    double speedup = (cpu_time.count() * 1000) / gpu_time;                      // Вычисляем на сколько раз GPU быстрее CPU
    cout << "GPU работает примерно в " << speedup << " раз быстрее CPU." << endl; // Выводим ускорение

    // Освобождение памяти
    delete[] h_array;               // Освобождаем память CPU
    cudaFree(d_array);              // Освобождаем память GPU массива
    cudaFree(d_result);             // Освобождаем память GPU результата

    return 0;                       // Завершение программы
}


Writing assignment41.cu


In [4]:
# Компиляция
!nvcc assignment41.cu -o assignment41 -arch=sm_75 -std=c++11            # -arch=sm_75  - архитектура GPU (Tesla T4 в Colab = sm_75)
                                                                        # -std=c++11 — стандарт C++
# Запуск
!./assignment41

CPU результат: 100000
GPU результат: 100000
CPU время: 0.279493 мс
GPU время: 0.103392 мс
GPU работает примерно в 2.70324 раз быстрее CPU.
